# Erstellen Sie ein Klassifikationsmodell: Köstliche asiatische und indische Küchen


## Küchenklassifikatoren 2

In dieser zweiten Lektion zur Klassifikation werden wir `weitere Möglichkeiten` zur Klassifikation von kategorialen Daten erkunden. Außerdem werden wir die Auswirkungen der Wahl eines Klassifikators gegenüber einem anderen kennenlernen.

### [**Quiz vor der Vorlesung**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Voraussetzungen**

Wir gehen davon aus, dass Sie die vorherigen Lektionen abgeschlossen haben, da wir einige zuvor erlernte Konzepte weiterführen werden.

Für diese Lektion benötigen wir die folgenden Pakete:

-   `tidyverse`: Das [tidyverse](https://www.tidyverse.org/) ist eine [Sammlung von R-Paketen](https://www.tidyverse.org/packages), die darauf abzielt, Datenwissenschaft schneller, einfacher und unterhaltsamer zu machen!

-   `tidymodels`: Das [tidymodels](https://www.tidymodels.org/) Framework ist eine [Sammlung von Paketen](https://www.tidymodels.org/packages/) für Modellierung und maschinelles Lernen.

-   `themis`: Das [themis-Paket](https://themis.tidymodels.org/) bietet zusätzliche Rezeptschritte für den Umgang mit unausgewogenen Daten.

Sie können diese Pakete wie folgt installieren:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativ überprüft das untenstehende Skript, ob Sie die für dieses Modul benötigten Pakete installiert haben, und installiert sie für Sie, falls sie fehlen.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Eine Klassifikationsübersicht**

In unserer [vorherigen Lektion](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) haben wir versucht, die Frage zu beantworten: Wie wählt man zwischen mehreren Modellen aus? In hohem Maße hängt dies von den Eigenschaften der Daten und der Art des Problems ab, das wir lösen möchten (zum Beispiel Klassifikation oder Regression?).

Zuvor haben wir die verschiedenen Möglichkeiten kennengelernt, die Ihnen zur Verfügung stehen, wenn Sie Daten mithilfe von Microsofts Spickzettel klassifizieren. Das Machine-Learning-Framework von Python, Scikit-learn, bietet einen ähnlichen, aber detaillierteren Spickzettel, der Ihnen dabei helfen kann, Ihre Auswahl an Schätzern (ein anderer Begriff für Klassifikatoren) weiter einzugrenzen:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tipp: [Besuchen Sie diese Karte online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) und klicken Sie entlang des Pfads, um die Dokumentation zu lesen.
>
> Die [Tidymodels-Referenzseite](https://www.tidymodels.org/find/parsnip/#models) bietet ebenfalls eine hervorragende Dokumentation über verschiedene Modelltypen.

### **Der Plan** 🗺️

Diese Karte ist sehr hilfreich, sobald Sie ein klares Verständnis Ihrer Daten haben, da Sie entlang ihrer Pfade zu einer Entscheidung „gehen“ können:

-   Wir haben \>50 Proben

-   Wir möchten eine Kategorie vorhersagen

-   Wir haben beschriftete Daten

-   Wir haben weniger als 100.000 Proben

-   ✨ Wir können einen Linear SVC wählen

-   Wenn das nicht funktioniert, da wir numerische Daten haben

    -   Können wir einen ✨ KNeighbors Classifier ausprobieren

        -   Wenn das nicht funktioniert, probieren Sie ✨ SVC und ✨ Ensemble Classifiers

Dies ist ein sehr hilfreicher Pfad, dem man folgen kann. Jetzt legen wir direkt los und nutzen das [tidymodels](https://www.tidymodels.org/) Modellierungs-Framework: eine konsistente und flexible Sammlung von R-Paketen, die entwickelt wurden, um gute statistische Praktiken zu fördern 😊.

## 2. Daten aufteilen und mit unausgewogenen Datensätzen umgehen.

Aus unseren vorherigen Lektionen haben wir gelernt, dass es eine Reihe von gemeinsamen Zutaten in unseren Küchen gab. Außerdem gab es eine ziemlich ungleiche Verteilung in der Anzahl der Küchen.

Wir werden damit umgehen, indem wir:

-   Die häufigsten Zutaten, die Verwirrung zwischen verschiedenen Küchen schaffen, mit `dplyr::select()` entfernen.

-   Ein `recipe` verwenden, das die Daten vorverarbeitet, um sie für die Modellierung vorzubereiten, indem ein `over-sampling`-Algorithmus angewendet wird.

Wir haben das oben bereits in der vorherigen Lektion behandelt, daher sollte das ein Kinderspiel sein 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Umgang mit unausgeglichenen Daten

Unausgeglichene Daten haben oft negative Auswirkungen auf die Modellleistung. Viele Modelle erzielen die besten Ergebnisse, wenn die Anzahl der Beobachtungen gleich ist, und haben daher Schwierigkeiten mit unausgeglichenen Daten.

Es gibt im Wesentlichen zwei Ansätze, um mit unausgeglichenen Datensätzen umzugehen:

-   Hinzufügen von Beobachtungen zur Minderheitsklasse: `Over-sampling`, z. B. mit einem SMOTE-Algorithmus, der synthetisch neue Beispiele der Minderheitsklasse generiert, indem er die nächsten Nachbarn dieser Fälle verwendet.

-   Entfernen von Beobachtungen aus der Mehrheitsklasse: `Under-sampling`

In unserer vorherigen Lektion haben wir demonstriert, wie man mit unausgeglichenen Datensätzen mithilfe eines `recipe` umgeht. Ein Recipe kann als eine Art Blaupause betrachtet werden, die beschreibt, welche Schritte auf einen Datensatz angewendet werden sollten, um ihn für die Datenanalyse vorzubereiten. In unserem Fall möchten wir eine gleichmäßige Verteilung der Anzahl unserer Küchenstile im `training set` erreichen. Legen wir los.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Jetzt sind wir bereit, Modelle zu trainieren 👩‍💻👨‍💻!

## 3. Über multinomiale Regressionsmodelle hinaus

In unserer vorherigen Lektion haben wir uns mit multinomialen Regressionsmodellen beschäftigt. Lassen Sie uns einige flexiblere Modelle für die Klassifikation erkunden.

### Support Vector Machines

Im Kontext der Klassifikation ist `Support Vector Machines` eine Technik des maschinellen Lernens, die versucht, eine *Hyperebene* zu finden, die die Klassen "optimal" trennt. Schauen wir uns ein einfaches Beispiel an:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ trennt die Klassen nicht. H2~ trennt sie, aber nur mit einem kleinen Abstand. H3~ trennt sie mit dem maximalen Abstand.

#### Linearer Support-Vector-Klassifikator

Support-Vector-Clustering (SVC) ist ein Mitglied der Familie der Support-Vector-Maschinen (SVM), einer Technik des maschinellen Lernens. Beim SVC wird die Hyperebene so gewählt, dass sie `die meisten` Trainingsbeobachtungen korrekt trennt, aber `einige` Beobachtungen möglicherweise falsch klassifiziert. Indem einige Punkte auf der falschen Seite erlaubt werden, wird die SVM robuster gegenüber Ausreißern und erzielt dadurch eine bessere Generalisierung auf neue Daten. Der Parameter, der diese Abweichung reguliert, wird als `cost` bezeichnet und hat standardmäßig den Wert 1 (siehe `help("svm_poly")`).

Lassen Sie uns einen linearen SVC erstellen, indem wir `degree = 1` in einem polynomialen SVM-Modell setzen.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nun, da wir die Vorverarbeitungsschritte und die Modellspezifikation in einem *Workflow* erfasst haben, können wir fortfahren, den linearen SVC zu trainieren und die Ergebnisse dabei auszuwerten. Für die Leistungskennzahlen erstellen wir ein Metrik-Set, das folgende Werte bewertet: `accuracy`, `sensitivity`, `Positive Predicted Value` und `F Measure`.

> `augment()` fügt der angegebenen Datenmenge Spalte(n) für Vorhersagen hinzu.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Die Support Vector Machine (SVM) ist eine Erweiterung des Support Vector Classifiers, um eine nicht-lineare Grenze zwischen den Klassen zu ermöglichen. Im Wesentlichen nutzen SVMs den *Kernel-Trick*, um den Merkmalsraum zu erweitern und sich an nichtlineare Beziehungen zwischen den Klassen anzupassen. Eine beliebte und äußerst flexible Kernel-Funktion, die von SVMs verwendet wird, ist die *Radial Basis Function.* Schauen wir uns an, wie sie sich auf unseren Daten schlägt.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Viel besser 🤩!

> ✅ Bitte siehe:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning mit R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), Eine Einführung in Statistisches Lernen mit Anwendungen in R
>
> für weiterführende Lektüre.

### Nächster-Nachbar-Klassifikatoren

Der *k*-nächste Nachbar (KNN) ist ein Algorithmus, bei dem jede Beobachtung basierend auf ihrer *Ähnlichkeit* zu anderen Beobachtungen vorhergesagt wird.

Lass uns einen auf unsere Daten anwenden.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Es scheint, dass dieses Modell nicht besonders gut abschneidet. Wahrscheinlich wird eine Anpassung der Modellparameter (siehe `help("nearest_neighbor")`) die Leistung verbessern. Probieren Sie es unbedingt aus.

> ✅ Weitere Informationen finden Sie unter:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> um mehr über *K*-Nearest Neighbors-Klassifikatoren zu erfahren.

### Ensemble-Klassifikatoren

Ensemble-Algorithmen funktionieren, indem sie mehrere Basis-Estimatoren kombinieren, um ein optimales Modell zu erstellen, entweder durch:

`bagging`: Anwenden einer *Mittelungsfunktion* auf eine Sammlung von Basismodellen

`boosting`: Erstellen einer Sequenz von Modellen, die aufeinander aufbauen, um die Vorhersageleistung zu verbessern.

Beginnen wir mit einem Random-Forest-Modell, das eine große Sammlung von Entscheidungsbäumen erstellt und dann eine Mittelungsfunktion anwendet, um ein besseres Gesamtmodell zu erhalten.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Gute Arbeit 👏!

Lass uns auch ein Boosted-Tree-Modell ausprobieren.

Boosted Tree definiert eine Ensemble-Methode, die eine Reihe von sequentiellen Entscheidungsbäumen erstellt, bei denen jeder Baum von den Ergebnissen der vorherigen Bäume abhängt, um den Fehler schrittweise zu reduzieren. Der Fokus liegt auf den Gewichten der falsch klassifizierten Elemente, und die Anpassung für den nächsten Klassifikator wird entsprechend korrigiert.

Es gibt verschiedene Möglichkeiten, dieses Modell anzupassen (siehe `help("boost_tree")`). In diesem Beispiel passen wir Boosted Trees über die `xgboost`-Engine an.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Bitte sehen Sie:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Untersucht das AdaBoost-Modell, das eine gute Alternative zu xgboost darstellt.
>
> um mehr über Ensemble-Klassifikatoren zu erfahren.

## 4. Extra - Vergleich mehrerer Modelle

Wir haben in diesem Lab eine ganze Reihe von Modellen angepasst 🙌. Es kann mühsam oder aufwendig werden, viele Workflows aus verschiedenen Sets von Preprozessoren und/oder Modellspezifikationen zu erstellen und dann die Leistungskennzahlen einzeln zu berechnen.

Schauen wir, ob wir das vereinfachen können, indem wir eine Funktion erstellen, die eine Liste von Workflows auf den Trainingssatz anpasst und dann die Leistungskennzahlen basierend auf dem Testsatz zurückgibt. Wir werden `map()` und `map_dfr()` aus dem [purrr](https://purrr.tidyverse.org/) Paket verwenden, um Funktionen auf jedes Element in einer Liste anzuwenden.

> [`map()`](https://purrr.tidyverse.org/reference/map.html)-Funktionen ermöglichen es, viele for-Schleifen durch Code zu ersetzen, der sowohl kürzer als auch leichter lesbar ist. Der beste Ort, um mehr über die [`map()`](https://purrr.tidyverse.org/reference/map.html)-Funktionen zu lernen, ist das [Kapitel über Iteration](http://r4ds.had.co.nz/iteration.html) in "R for Data Science".


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) Paket ermöglicht es Nutzern, eine große Anzahl von Modellen zu erstellen und einfach anzupassen. Es ist jedoch hauptsächlich darauf ausgelegt, mit Resampling-Techniken wie `cross-validation` zu arbeiten, eine Methode, die wir noch behandeln werden.

## **🚀Herausforderung**

Jede dieser Techniken hat eine Vielzahl von Parametern, die Sie anpassen können, wie zum Beispiel `cost` bei SVMs, `neighbors` bei KNN, `mtry` (zufällig ausgewählte Prädiktoren) bei Random Forest.

Recherchieren Sie die Standardparameter jedes Modells und überlegen Sie, was das Anpassen dieser Parameter für die Qualität des Modells bedeuten würde.

Um mehr über ein bestimmtes Modell und seine Parameter zu erfahren, verwenden Sie: `help("model")`, z. B. `help("rand_forest")`.

> In der Praxis *schätzen* wir normalerweise die *besten Werte* für diese Parameter, indem wir viele Modelle auf einem `simulierten Datensatz` trainieren und messen, wie gut diese Modelle abschneiden. Dieser Prozess wird **Tuning** genannt.

### [**Quiz nach der Vorlesung**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Überblick & Selbststudium**

Es gibt viele Fachbegriffe in diesen Lektionen, nehmen Sie sich daher einen Moment Zeit, um [diese Liste](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) nützlicher Begriffe zu überprüfen!

#### EIN DANKESCHÖN AN:

[`Allison Horst`](https://twitter.com/allison_horst/) für die großartigen Illustrationen, die R einladender und ansprechender machen. Weitere Illustrationen finden Sie in ihrer [Galerie](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) und [Jen Looper](https://www.twitter.com/jenlooper) für die Erstellung der ursprünglichen Python-Version dieses Moduls ♥️

Viel Spaß beim Lernen,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
